<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Vault Auto Unseal
Vault supports opt-in automatic unsealing via cloud technologies: AliCloud KMS, Amazon KMS, Azure Key Vault and Google Cloud KMS.  This feature delegates the unsealing process to trusted cloud providers to ease operation.

<img src=images/shamir_secret_sharing.png width=500 align="left">
<img src=images/aws-auto-unseal.png width=300 align="right">   

**Note:**  
* With Auto-unseal enabled, Shamir's key are NOT used to unseal the Vault server; however they still exist
* The cloud provider's key is used to protect the **master key**
* You still need Shamir's keys (**recovery keys**) for other operations: regenerate a root token, or unseal Vault

**Prerequisites:** 
* You need to have Vault binary  
* You need to have AWS KMS key (enabled) and  AWS ACCESS ID & SECRET KEY

## Vault Setup

In [ ]:
# cleanup data directory

/bin/rm -rf $HOME/demos/vault_cluster/vrd/data
mkdir -p $HOME/demos/vault_cluster/vrd/data

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
which vault
vault -version

### Populate Vault server configuration file (hcl or json format)
Configures:
* storage
* listener
* ui
* telemetry
* seal
* etc

In [ ]:
cat <<EOF > $HOME/demos/vault_cluster/vrd/vrd_config.hcl
storage "raft" {
  path = "$HOME/demos/vault_cluster/vrd/data"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8200"
  cluster_address = "0.0.0.0:8201"
  tls_disable = "true"
}
seal "awskms" {
  region = "ap-southeast-1"
  kms_key_id = "a95ba676-d102-42ba-90bb-65bd4a8306ba"
}
api_addr = "https://127.0.0.1:8200"
cluster_addr = "https://127.0.0.1:8201"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$HOME/demos/vault_cluster/vrd/data/plugins"
EOF

\
**EXECUTE the following command in SEPARATE window**

```
# make sure AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY are set beforehand
#
vault server -config=$HOME/demos/vault_cluster/vrd/vrd_config.hcl
```

In [ ]:
vault status

##### Notice the Seal Type, Initialized and Sealed status
When a Vault server is started, it starts in a **sealed** state and it does not know how to decrypt data.  Before any operation can be performed on the Vault, it must be unsealed.  

First run of Vault, you'll see these in the Vault's log because the Vault has not been initialized.
2020-12-27T10:50:13.382+0800 [INFO]  core: stored unseal keys supported, attempting fetch  
2020-12-27T10:50:13.382+0800 [WARN]  failed to unseal core: error="stored unseal keys are supported, but none were found"  

## Initialize Vault

Initialization is the process of preparing Vault's storage backend to receive data, which includes:
* Encryption key (data key) gets generated 
* Generates in memory master key and applies Shamir's secret sharing algorithm to disamble the mster key 
(Unseal keys are created)
* Initial root token is setup

Note: the initialization takes few seconds to complete.  In HA mode, Vault servers share the same storage backend so you need to initialize one Vault.

Unseal keys are not returned to the operator.  Recovery key shares returned instead.  
Recovery keys are used for the followings:  
	- Trigger an auto-unseal  
	- Rekey   
    - Root token generation  
    


In [ ]:
vault operator init -key-shares=3 -key-threshold=2 

In [ ]:
vault operator init -recovery-shares=3 -recovery-threshold=2 | tee ~/demos/vault_cluster/vrd/vault_init_output

In [ ]:
export RECOVERY_KEY1=$(cat ~/demos/vault_cluster/vrd/vault_init_output | sed -n -e '/Recovery Key 1/ s/.*\: *//p')
export RECOVERY_KEY2=$(cat ~/demos/vault_cluster/vrd/vault_init_output | sed -n -e '/Recovery Key 2/ s/.*\: *//p')
export RECOVERY_KEY3=$(cat ~/demos/vault_cluster/vrd/vault_init_output | sed -n -e '/Recovery Key 3/ s/.*\: *//p')
export VAULT_ROOT_TOKEN=$(cat ~/demos/vault_cluster/vrd/vault_init_output | sed -n -e '/Initial Root Token/ s/.*\: *//p')

echo "Initial Root Token: $VAULT_ROOT_TOKEN"
echo "Recovery Keys:"
echo $RECOVERY_KEY1
echo $RECOVERY_KEY2
echo $RECOVERY_KEY3

In [ ]:
vault status

#### By this time Vault is initialized, unsealed and ready to be used.

In [ ]:
vault login $VAULT_ROOT_TOKEN

In [ ]:
vault secrets enable kv

## Shutdown the Vault and Start it

Shutdown is going to seal Vault.  
   
Restart Vault by EXECUTE the following command in **SEPARATE window**

```
vault server -config=$HOME/demos/vault_cluster/vrd/vrd_config.hcl
```

In [ ]:
vault status

Note:  Vault is **Auto-unseal (sealed=false)**   
You should see the followings in the Vault log file   
==> Vault server configuration:

           AWS KMS KeyID: a95ba676-d102-42ba-90bb-65bd4a8306ba
          AWS KMS Region: ap-southeast-1
             Api Address: https://127.0.0.1:8200
                     Cgo: disabled
         Cluster Address: https://127.0.0.1:8201
              Go Version: go1.15.4
...   
2020-12-23T23:15:57.564+0800 [INFO]  core: vault is unsealed   
2020-12-23T23:15:57.564+0800 [INFO]  core: **unsealed with stored key**   
2020-12-23T23:15:57.564+0800 [INFO]  core: entering standby mode  

### Disable AWS KMS key
Here's what you should see when disabling KMS key

2020-12-24T12:57:26.032+0800 [INFO]  core: stored unseal keys supported, attempting fetch  
2020-12-24T12:57:26.094+0800 [WARN]  failed to unseal core: error="fetching stored unseal keys failed: failed to encrypt keys for storage: error decrypting data   encryption key: DisabledException: arn:aws:kms:ap-southeast-1:451256726182:key/a95ba676-d102-42ba-90bb-65bd4a8306ba is disabled."  
2020-12-24T12:57:31.099+0800 [INFO]  core: stored unseal keys supported, attempting fetch  

### Seal Vault

Seal the Vault is an ultimate **break glass** procedure.  For example, you suspect that someone has unauthorized access to Vault with wrong intention.

In [ ]:
vault login $VAULT_ROOT_TOKEN

In [ ]:
vault operator seal

---
## Unseal Vault
Unsealing  is the process of constructing the master key necessary to decrypt the data encryption keys and read the data.  It requires a threshold of unseal keys (see the Intialize Vault above). Unsealing is a manual process and can become tedious when you have multiple Vault clusters! Hint: explore/use Auto-Unseal feature.



In [ ]:
vault operator unseal $RECOVERY_KEY1

In [ ]:
vault operator unseal $RECOVERY_KEY2

In [ ]:
vault status

### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">